In [1]:
# Install the required package
# If running in a notebook or interactive environment, use the following:
# %pip install transformers[torch]
# %pip install "accelerate>=0.26.0"
# If running as a script, use: pip install huggingface_hub[hf_xet]


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset

c:\AI\Deepseek_finetuning\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = [
    {"text": "Patient: I have a headache.\nDoctor: Please take rest and stay hydrated."},
    {"text": "Patient: My stomach is upset.\nDoctor: Try a light diet and monitor your symptoms."},
    {"text": "Patient: I feel dizzy.\nDoctor: Make sure you sit down and drink some water."},
    {"text": "Patient: I have a sore throat.\nDoctor: Gargle with warm salt water and drink warm fluids."},
    {"text": "Patient: I am coughing a lot.\nDoctor: Take honey with warm water and avoid cold drinks."},
    {"text": "Patient: I feel very tired.\nDoctor: Ensure you get enough sleep and eat a balanced diet."},
    {"text": "Patient: I have body pain.\nDoctor: Try a warm bath and consider a mild pain reliever."},
    {"text": "Patient: I feel anxious.\nDoctor: Practice deep breathing and relaxation techniques."},
    {"text": "Patient: I have a mild fever.\nDoctor: Take paracetamol and drink plenty of fluids."},
    {"text": "Patient: I have back pain.\nDoctor: Maintain proper posture and apply a hot compress."},
    {"text": "Patient: I feel short of breath.\nDoctor: Please sit upright and take slow deep breaths."},
    {"text": "Patient: I have a cold.\nDoctor: Rest well and drink ginger tea with honey."},
    {"text": "Patient: I am not sleeping well.\nDoctor: Avoid caffeine late in the day and try relaxation before bed."},
    {"text": "Patient: I have chest discomfort.\nDoctor: Please avoid exertion and get a medical check immediately if it worsens."},
    {"text": "Patient: I feel weak after exercise.\nDoctor: Rehydrate with water and include electrolytes if needed."}
]

In [4]:
dataset = Dataset.from_list(data)

model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"  

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name)

In [5]:
def tokenize_function(examples):
    tokens = tokenizer(examples["text"],
                       truncation=True,
                       max_length=64,      
                       padding="max_length")
    tokens["labels"] = tokens["input_ids"].copy()  
    return tokens

In [6]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map: 100%|██████████| 15/15 [00:00<00:00, 391.00 examples/s]


In [7]:
# %pip install "accelerate>=0.26.0"

training_args = TrainingArguments(
    output_dir="./fast_finetune",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    max_steps=10,             
    logging_steps=1,
    no_cuda=True,             
    fp16=False               
)

c:\AI\Deepseek_finetuning\.venv\lib\site-packages\transformers\training_args.py:1583: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

max_steps is given, it will override any value given in num_train_epochs


In [9]:
trainer.train()

 10%|█         | 1/10 [00:37<05:36, 37.44s/it]

{'loss': 8.8946, 'grad_norm': inf, 'learning_rate': 4.5e-05, 'epoch': 0.07}


 20%|██        | 2/10 [01:04<04:08, 31.07s/it]

{'loss': 10.3001, 'grad_norm': inf, 'learning_rate': 4e-05, 'epoch': 0.13}


 30%|███       | 3/10 [01:41<03:56, 33.72s/it]

...

{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.67}


100%|██████████| 10/10 [07:09<00:00, 35.32s/it]

{'train_runtime': 429.5922, 'train_samples_per_second': 0.023, 'train_steps_per_second': 0.023, 'train_loss': 1.9194735527038573, 'epoch': 0.67}


100%|██████████| 10/10 [07:10<00:00, 43.01s/it]


TrainOutput(global_step=10, training_loss=1.9194735527038573, metrics={'train_runtime': 429.5922, 'train_samples_per_second': 0.023, 'train_steps_per_second': 0.023, 'total_flos': 4916780728320.0, 'train_loss': 1.9194735527038573, 'epoch': 0.6666666666666666})

In [10]:
trainer.save_model('./fast_finetuned')
tokenizer.save_pretrained('./fast_finetuned')

('./fast_finetuned\\tokenizer_config.json',
 './fast_finetuned\\special_tokens_map.json',
 './fast_finetuned\\tokenizer.json')

In [11]:
def generate_response(prompt, max_length = 64):
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(**inputs, max_length = max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [12]:
prompt= 'Patient: I have a headache'
print(("chatbot response:",generate_response(prompt) ))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


('chatbot response:', 'Patient: I have a headache!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
